## Membros

- Amanda Oliveira
- Daniel Henrique
- Samuel Pedro

## Resumo

### Parâmetros

Será usado um kernel `rbf` em um kernel SVC. O `C` será otimizado pelo Optuna a fim de obter um resultado melhor. O `gamma` foi deixado no valor padrão que é `scale`, a fórmula referente a esse valor está disponível na documentação do scikit-learn.

### Representação adotada

A representação adotada foi escalar os valores númericos com um scaler do sklearn (no momento, um RobustScaler). Criar uma bag of words a partir do resumo. Criar uma bag of items com o elenco. Extrair o ano da data de estreia. E, por fim, descartar a história original e o titulo por não serem tão relevantes para os resultados. A coluna "adulo" foi deixada mesmo estando distribuida de maneira extremamente desuniforme, assumindo-se que o modelo irá incorporá-la melhor caso ela seja distribuída mais uniformemente.

Porém, o resultado mais efeitvo usa apenas a bag of words e a bag of items, descartando os demais fatores. Isso provavelmente está relacionado com a grande ausência de informações (NaNs) dos demais parâmetros.

### Método adotado

O método adotado foi um classificador SVC com um kernel `rbf`, já que sua acurácia se mostrou maior que o `linear`. O kernel `poly` não foi testado por falta de tempo, mas espera-se que ele seja menos preciso em grais baixos ou consideravelmente mais lento em grais altos. 

Os resultados do método Random Forest são levemente inferiores, mas conseguem incorporar mais informações para a predição.

## Pós-análise

Após algumas reflexões, percebeu-se que o modelo era altamente dependente ao resumo, o que não é algo necessariamente ruim. Por conta disso, decidimos adotar uma representação que consiste basicamente do resumo com uma limpeza aplicada por cima. Para os filmes que não se têm um resumo, usaremos um segundo modelo baseado em RandomForest para prever a categoria baseando-se nos demais parâmetros, caso esse modelo mostre-se insuficientemente preciso, substituiremos-o por uma escolha fixa do gênero comédia, já que esse é o valor estatisticamente mais comum para esse caso de borda em específico.

In [11]:
import pandas as pd
import numpy as np
from competicao_am.gerar_resultado_teste import gerar_saida_teste

In [12]:
df_treino = pd.read_csv('datasets/movies_amostra.csv')
df_predict = pd.read_csv('datasets/movies_amostra_teste_ex.csv')

df_treino.head(2)

,id,titulo,adulto,orcamento,idioma_original,popularidade,data_de_estreia,resumo,receita,duracao,genero,ator_1,ator_2,ator_3,ator_4,ator_5,dirigido_por,escrito_por_1,escrito_por_2,historia_original
0,86295,Treasure of the Yankee Zephyr,False,0,en,2.0,1981-11-28,In a lake high in the mountains of New Zealand...,0.0,108.0,Action,Ken Wahl,Lesley Ann Warren,Donald Pleasence,George Peppard,Bruno Lawrence,David Hemmings,Everett De Roche,NaN,NaN
1,289198,Redeemer,False,0,es,2.0,2014-09-18,A former hit-man for a drug cartel becomes a v...,0.0,88.0,Action,Marko Zaror,Loreto Aravena,Mauricio Diocares,José Luís Mósca,Noah Segan,Ernesto Díaz Espinoza,NaN,NaN,NaN


In [13]:
from competicao_am.preprocessamento_atributos_competicao import DataframePreprocessing
preprocessor = DataframePreprocessing(df_treino, df_treino, 'genero')

## Otimização dos parâmetros

In [14]:
import optuna
import pickle # Serialização dos resultados

In [15]:
import matplotlib.pyplot as plt
from sklearn import svm

from base_am.resultado import Fold
from base_am.avaliacao import Experimento
import competicao_am.metodo_competicao as mc
import competicao_am.avaliacao_competicao as ac

import json

## Um pouco sobre SVC's

### Kernel

O `kernel` indica o tipo de "hiper-plano" usado para separar as classes.

Os kernels a serem considerados são: `linear`, `rbf` e `poly`

#### `poly`: degree

Grau do polinômio a ser usado. Quanto maior o valor, maior o tempo gasto para o treino.

### gamma

Parâmetro usado para "hiper-planos" não lineares. Quanto maior, mais ele tenta en encaixar o modelo perfeitamente.

Valores muito altos caminhão em direção de um _overfitting_.

### C

C é o parâmetro de penalidade do erro. Ele controla o prejuízo entre paredes de decisões suaves e classificar os pontos corretamente.



Links úteis:

- [A guide to SVM parameter tuning](https://towardsdatascience.com/a-guide-to-svm-parameter-tuning-8bfe6b8a452c)

In [16]:
def experimento(metodo: mc.MetodoCompeticaoValidacaoSVM, num_trials: int = 2, otimizador:ac.OtimizacaoObjetivo = ac.OtimizacaoObjetivoSVCRbfComGamma, ml_method = None) -> Experimento:
    folds = Fold.gerar_k_folds(df_treino, val_k=5, col_classe="genero",
                                num_repeticoes=1, num_folds_validacao=4, num_repeticoes_validacao=1)

    ClasseObjetivo = ac.WrapperOtimizacaoObjetivo(otimizador, metodo=metodo)

    experimento = Experimento(folds, ml_method=ml_method, ClasseObjetivoOtimizacao=ClasseObjetivo, num_trials=num_trials)

    return experimento

def best_parameters(experiment: Experimento):
    results = []
    for study in experiment.studies_per_fold:
        results.append((study.best_trial.value, study.best_trial.params))
    results.sort()
    return list(reversed(sorted(results)))

In [17]:
import pickle
import os

g_version = 2

def do_experiment(name: str, metodo: mc.MetodoCompeticaoValidacaoSVM = None, num_trials: int = None, otimizador:ac.OtimizacaoObjetivo = None, ml_method = None, force: bool = False, version: int = g_version) -> Experimento:
    file_name = f'{name}__{version}.p'
    file_path = f'resultados/{file_name}'

    if not force and os.path.isfile(file_path):
        exp = None

        with open(file_path, 'rb') as exp_p:
            exp = pickle.load(exp_p)
            # Check instance here

        return exp
    
    if ml_method is None and (metodo is None or num_trials is None or otimizador is None):
        raise ValueError('To run the experiment, the appropriate parameters should be give (ml_method or metodo, num_trials and otimizador)')

    exp = experimento(metodo=metodo, num_trials=num_trials, otimizador=otimizador, ml_method=ml_method)
    macro_f1 = exp.macro_f1_avg

    with open(file_path, 'wb') as exp_p:
        # exp = pickle.dump(exp, exp_p)
        pass

    return exp

In [18]:
exp_test_rf = do_experiment('exp_test_rf', mc.MetodoCompeticaoValidacaoRF, 1000, ac.OtimizacaoObjetivoRandomForest, version=2)

In [19]:
exp_test_svm = do_experiment('exp_test_svm', mc.MetodoCompeticaoValidacaoSVM, 100, ac.OtimizacaoObjetivoSVCRbfComGamma, version=2)

In [20]:
print(json.dumps(best_parameters(exp_test_rf), indent=4))
print(json.dumps(best_parameters(exp_test_svm), indent=4))

[
    [
        0.596445251683727,
        {
            "min_samples_split": 0.04466339369901853,
            "max_features": 0.0010292429905777445,
            "num_arvores": 1
        }
    ],
    [
        0.5954073820857066,
        {
            "min_samples_split": 0.08842136011502941,
            "max_features": 0.22848035048457271,
            "num_arvores": 1
        }
    ],
    [
        0.5923505029598322,
        {
            "min_samples_split": 0.10251976896112781,
            "max_features": 0.4785340513058907,
            "num_arvores": 1
        }
    ],
    [
        0.5886801037590487,
        {
            "min_samples_split": 0.12650146532653397,
            "max_features": 0.32474851683945516,
            "num_arvores": 2
        }
    ],
    [
        0.5874204938135473,
        {
            "min_samples_split": 0.1164735770300016,
            "max_features": 0.45935262471729577,
            "num_arvores": 1
        }
    ]
]
[
    [
        0.784123350272648